# L6: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [6]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [8]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [9]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [10]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [11]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [12]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [13]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [14]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [15]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to ensure that the team is effectively monitoring and analyzing market data for AAPL to identify trends and predict market movements. I should delegate tasks and ask questions to gather the necessary insights and alerts about significant market opportunities or threats for AAPL.

Action: Delegate work to co-worker
Action Input: {"coworker": "Data Analyst", "task": "Continuously monitor and analyze market data for AAPL using statistical modeling and machine learning techniques", "context": "The goal is to identify trends and predict market movements for AAPL"}

> Entering new CrewAgentExecutor chain...
I need to gather real-time market data for AAPL to identify trends and predict movements. I should start by

Final Answer: 
Apple Inc. (AAPL) Stock Price, News, Quote & History - Yahoo Finance
AAPL Apple Inc. 223.87 +0.05% Summary News Research Chart Community Statistics Historical Data Profile Financials Analysis Options Holders Sustainability Unlock stock picks and a broker-level newsfeed that powers Wall Street. Upgrade Now NasdaqGS - Nasdaq Real Time Price • USD Apple Inc. (AAPL) Follow Compare Time to buy AAPL? 223.75 +3.02 +(1.37%) At close: March 25 at 4:00:02 PM EDT 223.87 +0.12 +(0.05%) Pre-Market: 4:20:00 AM EDT Time to buy AAPL? 1D 5D 5.20% 1M -9.43% 6M -1.16% YTD -10.65% 1Y 30.96% 5Y 264.53% All 174,230.74% Key Events Mountain Line Candle Baseline Mountain Bar Advanced Chart Loading Chart for AAPL Previous Close 220.73 Open 220.81 Bid 209.57 x 100 Ask 231.78 x 100 Day's Range 220.08 - 224.10 52 Week Range 164.08 - 260.10 Volume 34,299,912 Avg. Volume 52,855,600 Market Cap (intraday) 3.361T Beta (5Y Monthly) 1.18 PE Ratio (TTM) 35.46 EPS (TTM) 6.31 Earnings Date Apr 30, 2025 - May 

Final Answer: 
The Data Analyst has been delegated the task of continuously monitoring and analyzing market data for AAPL using statistical modeling and machine learning techniques to identify trends and predict market movements. The information gathered includes the current stock price, historical data, key events, financial highlights, research analysis, and company insights for Apple Inc. The Data Analyst will use this information to provide insights and alerts about significant market opportunities or threats for AAPL.

> Finished chain.
 [DEBUG]: [Crew Manager] Task output: The Data Analyst has been delegated the task of continuously monitoring and analyzing market data for AAPL using statistical modeling and machine learning techniques to identify trends and predict market movements. The information gathered includes the current stock price, historical data, key events, financial highlights, research analysis, and company insights for Apple Inc. The Data Analyst will use this inf

Final Answer: Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, Mac, iPad, and various subscription-based services. It operates platforms like the App Store and Apple Music. Apple serves consumers, businesses, education, and government markets. The company sells products through retail stores, online stores, and third-party channels. Apple's recent performance shows positive total returns and strong financial metrics.

> Finished chain.
 

Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, Mac, iPad, and various subscription-based services. It operates platforms like the App Store and Apple Music. Apple serves consumers, businesses, education, and government markets. The company sells products through retail stores, online stores, and third-party channels. Apple's recent performance s

Final Answer: marketwatch.comPlease enable JS and disable any ad blocker

> Finished chain.
 

marketwatch.comPlease enable JS and disable any ad blocker

Final Answer: Based on the insights provided by the Data Analyst on market data for AAPL and the user's specified medium risk tolerance level for trading AAPL, the potential trading strategies for AAPL that align with the user's risk tolerance are as follows:

1. Long-term investment in AAPL stock: Considering Apple Inc.'s strong financial metrics and positive total returns, a long-term investment strategy could be suitable for the user's medium risk tolerance level.

2. Covered call options strategy: Utilizing options trading with covered calls on AAPL stock could provide a balance of risk and potential returns for the user within their defined risk tolerance.

3. Dollar-cost averaging approach: Implementing a dollar-cost averaging strategy for regular purchases of AAPL stock over time could help mitigate risk and align with the use

Final Answer: 

Risk Analysis Report for Proposed Trading Strategies for AAPL:

1. Long-term investment in AAPL stock:
- Risk: Market volatility and fluctuations in the tech industry can impact the stock price.
- Mitigation: Diversification of the portfolio to reduce exposure to AAPL, setting stop-loss orders, and staying informed about market trends.

2. Covered call options strategy:
- Risk: Limited upside potential if the stock price surpasses the call option strike price.
- Mitigation: Implementing proper risk management techniques, understanding the options market, and using strategies like protective puts.

3. Dollar-cost averaging approach:
- Risk: Market timing risk and potential losses if the stock price declines consistently.
- Mitigation: Setting a consistent investment schedule, diversifying the investment across different time periods, and staying focused on long-term goals.

4. Diversified portfolio with AAPL exposure:
- Risk: Concentration risk if AAPL underperforms comp

- Display the final result as Markdown.

In [16]:
from IPython.display import Markdown
Markdown(result)

Risk Analysis Report for Proposed Trading Strategies for AAPL:

1. Long-term investment in AAPL stock:
- Risk: Market volatility and fluctuations in the tech industry can impact the stock price.
- Mitigation: Diversification of the portfolio to reduce exposure to AAPL, setting stop-loss orders, and staying informed about market trends.

2. Covered call options strategy:
- Risk: Limited upside potential if the stock price surpasses the call option strike price.
- Mitigation: Implementing proper risk management techniques, understanding the options market, and using strategies like protective puts.

3. Dollar-cost averaging approach:
- Risk: Market timing risk and potential losses if the stock price declines consistently.
- Mitigation: Setting a consistent investment schedule, diversifying the investment across different time periods, and staying focused on long-term goals.

4. Diversified portfolio with AAPL exposure:
- Risk: Concentration risk if AAPL underperforms compared to other assets in the portfolio.
- Mitigation: Regular portfolio rebalancing, diversifying across various asset classes, and monitoring the performance of AAPL relative to other investments.

Overall, it's crucial to consider the risk factors associated with each trading strategy and implement appropriate mitigation strategies to align with the user's medium risk tolerance level and investment goals. By staying informed, diversifying the portfolio, and managing risks effectively, the user can optimize their trading outcomes and achieve their desired financial objectives.